## Fabric Optimize spark for query analysis

In [1]:
print(1)

1


In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

df = spark.read.format("csv") \
               .option("header","true") \
               .option("inferschema","true") \
               .load("Files/csv/FileOptimization/ecommerce_transactions.csv")

# df now is a Spark DataFrame containing CSV data from "Files/csv/ecommerce_transactions.csv".
display(df.limit(5))

StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f9e14378-34bf-4f89-96ef-90ad91ecd924)

In [3]:
display(df.count())

StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 5, Finished, Available, Finished)

5000000

**Creating a table**

In [4]:
df.write.format("delta").mode("overwrite").saveAsTable("EcomSales")

StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 6, Finished, Available, Finished)

In [5]:
%%pyspark
print(spark.conf.get('spark.sql.parquet.vorder.default'))
print(spark.conf.get("spark.databricks.delta.optimizeWrite.enabled"))

StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 7, Finished, Available, Finished)

false
None


In [6]:
from pyspark.sql.functions import year, to_timestamp, col

# Read the Delta table
df = spark.read.table("EcomSales")

# Convert string to TimestampType
df = df.withColumn("Timestamp", to_timestamp(col("Timestamp"), "M/d/yyyy H:mm"))

# Add a Year column extracted from Timestamp
df = df.withColumn("Year", year(col("Timestamp")))

# Aggregate by Year across all products
result_df = df.groupBy("Year").agg({"Amount": "sum"})

display(result_df)


StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 474c4c47-2c8b-44b5-b487-b33f1047cd44)

In [7]:
# Aggregate by Year across all products
result_df = df.groupBy("Year").agg({"Amount": "sum"}).explain(mode="cost") 

StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 9, Finished, Available, Finished)

== Optimized Logical Plan ==
Aggregate [Year#1796], [Year#1796, sum(Amount#1779) AS sum(Amount)#2266], Statistics(sizeInBytes=95.4 MiB)
+- Project [Amount#1779, year(cast(gettimestamp(Timestamp#1781, M/d/yyyy H:mm, TimestampType, Some(UTC), false) as date)) AS Year#1796], Statistics(sizeInBytes=95.4 MiB, rowCount=5.00E+6)
   +- Relation spark_catalog.chimcobldhq2aj39epik8pbddtpiaj2860oiap32ds.ecomsales[TransactionID#1776,UserID#1777,ProductCategory#1778,Amount#1779,Country#1780,Timestamp#1781] parquet, Statistics(sizeInBytes=476.8 MiB, rowCount=5.00E+6)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Year#1796], functions=[sum(Amount#1779)], output=[Year#1796, sum(Amount)#2266])
   +- Exchange hashpartitioning(Year#1796, 200), ENSURE_REQUIREMENTS, [plan_id=850]
      +- HashAggregate(keys=[Year#1796], functions=[partial_sum(Amount#1779)], output=[Year#1796, sum#2271])
         +- Project [Amount#1779, year(cast(gettimestamp(Timestamp#1781, M/d/yyyy H:mm,

In [8]:
spark.conf.set("spark.sql.parquet.vorder.default", "true")
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")

StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 10, Finished, Available, Finished)

In [9]:
%%pyspark
print(spark.conf.get('spark.sql.parquet.vorder.default'))
print(spark.conf.get("spark.databricks.delta.optimizeWrite.enabled"))

StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 11, Finished, Available, Finished)

true
true


In [10]:
df.write.format("delta").mode("overwrite").saveAsTable("EcomSales2")

StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 12, Finished, Available, Finished)

In [11]:
from pyspark.sql.functions import year, to_timestamp, col

# Read the Delta table
df = spark.read.table("EcomSales")

# Convert string to TimestampType
df = df.withColumn("Timestamp", to_timestamp(col("Timestamp"), "M/d/yyyy H:mm"))

# Add a Year column extracted from Timestamp
df = df.withColumn("Year", year(col("Timestamp")))

# Aggregate by Year across all products
result_df = df.groupBy("Year").agg({"Amount": "sum"})

display(result_df)


StatementMeta(, bdd26204-5372-49bd-acd6-28a4322d1186, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ee675711-b4e5-4646-9825-1064ca1eeb43)

In [ ]:
# Aggregate by Year across all products
result_df = df.groupBy("Year").agg({"Amount": "sum"}).explain(mode="cost") 